In [94]:
using AutomotiveDrivingModels
using AutoViz
using AutomotiveSensors
using AutomotivePOMDPs
using Parameters
using Reel
using StaticArrays
using ProfileView
using Revise

In [102]:
include("../src/pedestrian_crossing/frenet_pedestrian_emergency_system.jl")

getObjectConfidenceInterval (generic function with 1 method)

In [106]:
params = CrosswalkParams()
params.obstacles_visible = true

obstacle_1 = ConvexPolygon([VecE2(40, -1.5), VecE2(40, -4.5), VecE2(46.5, -4.5), VecE2(46.5, -1.5)],4)
obstacle_2 = ConvexPolygon([VecE2(40, +4.5), VecE2(40, +7.5), VecE2(46.5, +7.5), VecE2(46.5, +4.5)],4)
params.obstacles = [obstacle_1, obstacle_2]
#params.obstacles = [obstacle_2]

env = CrosswalkEnv(params);



ego_v = 10.0
ped_v = 5.0/3.6
hitpoint = 50             # 50: middle, 100: left corner, 0: right corner
ped_theta = π/2


# fix values
ped_x = 50.0
ped_y_start = -2
ego_y = 0


ped_y_offset_hitpoint = VehicleDef().width * hitpoint / 100 - VehicleDef().width/2
ped_y_end = ped_y_offset_hitpoint
ped_t_collision = (ped_y_end - ped_y_start) / ped_v; 
ego_x = ped_x - ego_v * ped_t_collision - VehicleDef().length/2;


# Car definition
car_initial_state = VehicleState(VecSE2(ego_x, ego_y, 0.), env.roadway.segments[1].lanes[1], env.roadway, ego_v)
car = Vehicle(car_initial_state, VehicleDef(), 1)

# Pedestrian definition using our new Vehicle definition
ped_initial_state = VehicleState(VecSE2(ped_x,ped_y_start,ped_theta), env.crosswalk, env.roadway, ped_v)
ped = Vehicle(ped_initial_state, AutomotivePOMDPs.PEDESTRIAN_DEF, 2)

scene = Scene()
push!(scene, car)
push!(scene, ped)



cam = FitToContentCamera(0.);
timestep = 0.05

pos_noise = 0.1
vel_noise = 0.1
false_positive_rate = 0.05
false_negative_rate = 0.01
rng = MersenneTwister(1);
sensor = AutomotiveSensors.GaussianSensor(AutomotiveSensors.LinearNoise(10, pos_noise, 0.00), 
                 AutomotiveSensors.LinearNoise(10, vel_noise, 0.00), false_positive_rate, false_negative_rate, rng) 



# define a model for each entities present in the scene
models = Dict{Int, DriverModel}()

ego_id = 1
ped_id = 2
models[ego_id] = EmergencySystem(a=LatLonAccel(0.0, 0.0),env=env,sensor=sensor, obstacles=env.obstacles, timestep=timestep)
models[ped_id] = ConstantPedestrian(v_desired=ped_v, dawdling_amp=0.05) # dumb model


nticks = 80
rec = SceneRecord(nticks+1, timestep)
# execute the simulation

risk = Float64[]
collision_rate = Float64[]
ttc = Float64[]
brake_request = Bool[]
prediction = Vector{Array{Float64}}()

sensor_observations = [Vehicle[]]

obs_callback = (ObservationCallback(risk, collision_rate, ttc, brake_request, prediction,sensor_observations),)

tic()
simulate!(rec, scene, env.roadway, models, nticks, obs_callback)
toc()


TTB: 0.885
standby
TTB: 0.885
standby
TTB: 0.885 collision_rate: 1.0 ttc_m: 1.3689720134952392 ttc_std: 0.04978641180991287Risk: 0.6971802149069114 ttc_min: 1.2693991898754136

standby
TTB: 0.885 collision_rate: 0.7 ttc_m: 1.3477266423189307 ttc_std: 0.054448679192948025Risk: 0.714384145965861 ttc_min: 1.2388292839330346

standby
TTB: 0.885 collision_rate: 0.7 ttc_m: 1.2877516093872434 ttc_std: 0.05302827886617544Risk: 0.7489241820557774 ttc_min: 1.1816950516548925

standby
TTB: 0.885 collision_rate: 0.7 ttc_m: 1.2276840552033998 ttc_std: 0.051607878539402914Risk: 0.7870386399296602 ttc_min: 1.124468298124594

standby
TTB: 0.885 collision_rate: 0.9 ttc_m: 1.1905778895318222 ttc_std: 0.048637540844627215Risk: 0.8094738197429356 ttc_min: 1.0933028078425677

standby
TTB: 0.885 collision_rate: 1.0 ttc_m: 1.1408752506550734 ttc_std: 0.04485068994944739Risk: 0.8419159043245255 ttc_min: 1.0511738707561786

standby
TTB: 0.885 collision_rate: 1.0 ttc_m: 1.0783941983864274 ttc_std: 0.04356311033

1.393311347

In [107]:
duration, fps, render_hist = animate_record(rec, timestep, env, sensor, sensor_observations, risk, ttc, collision_rate, brake_request, prediction)
film = roll(render_hist, fps = fps, duration = duration)

Reel.Frames{MIME{Symbol("image/png")}}("/tmp/tmpVGCVDA", 0x0000000000000050, 20.0, nothing)